### Hyun-Joon Yang
### yanghyun@usc.edu
### QBIO 401
### Assignment 5

We are going to write a UPGMA algorithm in Python and analyze some human data.

On Blackboard, you can find a file named upgmaData.py, which contains:
1. humansList (and testList) are species lists. They are in the form of a tuple.
2. humansDistances (and testDistances) are dictionaries specifying pairwise distances between species. They are in the form where the key is a tuple which is a pair of species name, e.g., (“species A”,”species B”) and the value is the pairwise distance. The following are the details for the human data:

Modern human data: distances based on mitochondrial sequence
<table>
    <tr>
        <td>San</td>
        <td>San individual from southern Africa</td>
    </tr>
    <tr>
        <td>Yoruba</td>
        <td>Yoruba individual from western Africa</td>
    </tr>
    <tr>
        <td>Finnish</td>
        <td>Finnish individual from northern Europe</td>
    </tr>
    <tr>
        <td>Kikuyu</td>
        <td>Kikuyu individual from eastern Africa</td>
    </tr>
    <tr>
        <td>Papuan</td>
        <td>Papuan individual from New Guinea</td>
    </tr>
    <tr>
        <td>Han</td>
        <td>Han individual from China</td>
    </tr>
</table>

Note: use “from upgmaData import \*” on the first line of your code to be able to use the data.

In [1]:
from upgmaData import *

### Write the following Python class for a tree node (as taught in Jinsen’s discussion):
#### 1. TreeNode

* Has four properties including key, distance, left and right, where key is for storing species name, distance is for branch length, left is for the left child and right is for the right child. The class has a function print to print the tree structure in the form of (key, distance, left, right). For example, (‘W_X’,0.5,(‘W’,0,(),()),(‘X’,0,(),())) is for the tree W_X with left child (‘W’,0,(),()) and right child (‘X’,0,(),()); the branch length to its child is 0.5.

In [2]:
class TreeNode:
    def __init__(self, key, distance, left, right):
        self.key = key
        self.distance = distance
        self.left = left
        self.right = right
        if left is None and right is None:
            self.leaves = 1
        elif left is None:
            self.leaves = self.right.leaves
        elif right is None:
            self.leaves = self.left.leaves
        else:
            self.leaves = self.left.leaves + self.right.leaves
        
    def print_helper(self):
        if self.left is None and self.right is None:
            return (self.key, self.distance, (), ())
        elif self.left is None:
            return (self.key, self.distance, (), self.right.print_helper())
        elif self.right is None:
            return (self.key, self.distance, self.left.print_helper(), ())
        return (self.key, self.distance, self.left.print_helper(), self.right.print_helper())
        
    def print(self):
        print(self.print_helper())

In [3]:
a = TreeNode('W', 0, None, None)
a.print()

('W', 0, (), ())


In [4]:
W = TreeNode('W', 0, None, None)
X = TreeNode('X', 0, None, None)
W_X = TreeNode('W_X', 0.5, W, X)
W_X.print()

('W_X', 0.5, ('W', 0, (), ()), ('X', 0, (), ()))


In [5]:
W = TreeNode('W', 0, None, None)
X = TreeNode('X', 0, None, None)
W_X = TreeNode('W_X', 0.5, W, X)
print(W_X.leaves, W.leaves, X.leaves)

2 1 1


### Write the following functions:
#### 2. findClosestPair(Distances)
* Inputs Distances dictionary and returns the key for the pair of nodes that are closest. For example, if the input is testDistances, the function returns (‘W’, ‘X’).

In [6]:
def findClosestPair(distances):
    """
    takes dictionary of distances and returns the key for the pair of nodes that are closest
    :param distance: dictionary of distances
    :return: key for the pair of nodes that are closest
    """
    key_min = list(distances.keys())[0]
    for key in distances:
        if distances[key] < distances[key_min]:
            key_min = key
    return key_min

In [7]:
print(type(findClosestPair(testDistances)))
print(findClosestPair(testDistances))

<class 'tuple'>
('W', 'X')


#### 3. updateDist(Distances, newNode)
* Inputs Distances dictionary and a new tree node. Updates the Distances dictionary by adding the distances between newNode and all the other nodes. Does not calculate the distance between newNode and its children in the Distances. This function returns the updated Distances dictionary.

In [8]:
def calcDist(new_node, node, distances):
    """
    calculates the distance between a newly made node and another node
    :param new_node: new TreeNode
    :param node: TreeNode
    :param distances: dictionary containing all the distances of parentless nodes (aside from children of new node)
    :return: distance between the new node and another node
    """
    key = (new_node.key, node.key)
    if key in distances:
        return distances[key]
    left = new_node.left
    right = new_node.right
    if node == left or node == right:
        return distances[(left.key, right.key)] / 2
    distance_left = distances[(left.key, node.key)] * left.leaves
    distance_right = distances[(right.key, node.key)] * right.leaves
    distance = (distance_left + distance_right) / new_node.leaves
    return distance

In [9]:
def updateDist(distances, new_node, nodes):
    """
    updates the dictionary by adding the distances between the new node and all the other nodes
    and removing old distances
    :param distances: dictionary of distances
    :param new_node: newly made TreeNode
    :param nodes: dictionary that maps keys to TreeNodes that do not have parent
    :return: updated dictionary
    """
    left = new_node.left.key
    right = new_node.right.key
    
    # add new distances
    for key in nodes:
        distance = calcDist(new_node, nodes[key], distances)
        distances[(new_node.key, key)] = distance
        distances[(key, new_node.key)] = distance
    # remove old distances
    keys = list(distances.keys())
    for key in keys:
        if new_node.key in key:
            continue
        if key[0] not in nodes or key[1] not in nodes:
            del distances[key]
    return distances

In [10]:
W = TreeNode('W', 0, None, None)
X = TreeNode('X', 0, None, None)
W_X = TreeNode('W_X', -1, W, X)
dist_left = calcDist(W_X, W_X.left, testDistances)
dist_right = calcDist(W_X, W_X.right, testDistances)
assert dist_left == dist_right
W_X.distance = dist_left
print(W_X.distance)

0.5


#### 4. upgma(Distances)
* This function runs the UPGMA algorithm described in lecture. First, it initializes a list TreeNodesList with multiple new TreeNode referring to humansList (the list TreeNodesList should contain six TreeNodes). Then, it repeats the following steps until there is only one tree node left in the list TreeNodesList, at which point this tree is returned.

    1. Find the key for the closest pair of nodes in Distances (use findClosestPair)
    2. Create a new TreeNode and assign the pair of nodes stored in TreeNodesList as its children. Calculate the distance between the new TreeNode to its children and store the value in the new TreeNode.
    3. Update the TreeNodesList by removing the pair of nodes referring to 1.
    4. Update the Distances dictionary (use updateDist)
    5. Add the new TreeNode into TreeNodesList.

In [11]:
def upgma(distances, leafList):
    # dict for all the nodes that do not have a parent
    # maps key to node
    nodes = {}
    # create TreeNodes for all leaf nodes
    for leaf in leafList:
        node = TreeNode(leaf, 0, None, None)
        nodes[leaf] = node
    # remove all distances to self
    keys = list(distances.keys())
    for key in keys:
        if key[0] == key[1]:
            del distances[key]
    # add to tree using upgma until root reached
    while len(nodes) > 1:
        # instantiate new parent node
        key_min = findClosestPair(distances)
        key = '(' + key_min[0] + ',' + key_min[1] +')'
        left = nodes[key_min[0]]
        right = nodes[key_min[1]]
        new_node = TreeNode(key, -1, left, right)
        # calculate distance to children
        dist_left = calcDist(new_node, new_node.left, distances)
        dist_right = calcDist(new_node, new_node.right, distances)
#         print(new_node.key, new_node.left.key, new_node.right.key, dist_left, dist_right)
        assert dist_left == dist_right
        new_node.distance = dist_left
        # remove children from nodes dict
        del nodes[key_min[0]]
        del nodes[key_min[1]]
        # get updated distances & remove distances of children
        distances = updateDist(distances, new_node, nodes)
        # add new node into nodes dict
        nodes[key] = new_node
#         print(nodes.keys())
#         print(distances)
    # return root
    key_root = list(nodes.keys())[0]
    return nodes[key_root]

In [12]:
root = upgma(testDistances, testList)
root.print()

('((Y,Z),(W,X))', 1.25, ('(Y,Z)', 0.5, ('Y', 0, (), ()), ('Z', 0, (), ())), ('(W,X)', 0.5, ('W', 0, (), ()), ('X', 0, (), ())))


### Use aforementioned functions to analyze human data:
#### 5. Use upgma to generate one tree using humansList and humansDistances. Turn in your code, the output the code for humans, and explain what you find from the results

In [13]:
root = upgma(humansDistances, humansList)
root.print()

('(((((Finnish,Papuan),Han),Yoruba),Kikuyu),San)', 0.0026804, ('((((Finnish,Papuan),Han),Yoruba),Kikuyu)', 0.0023785, ('(((Finnish,Papuan),Han),Yoruba)', 0.0010831666666666665, ('((Finnish,Papuan),Han)', 0.0009215, ('(Finnish,Papuan)', 0.000656, ('Finnish', 0, (), ()), ('Papuan', 0, (), ())), ('Han', 0, (), ())), ('Yoruba', 0, (), ())), ('Kikuyu', 0, (), ())), ('San', 0, (), ()))


It seems like the Finnish and Papuan are the closest with Han, Yoruba, Kikuyu, and San following in that order. This is surprising since I would expect the different African people to be the closest where as people from different continents would differ the most.